# Check predictions

Creates an excel file with all predictions that have the confidence score for target object higher than 0.5

The excel file contains the following columns:
- ImageName
- BoundingBox
- lat
- lon
- yaw
- timestampImage
- timestampDetection

In [ ]:
import json
import glob
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from draw_utils import get_exif_data, get_lat_lon
from PIL import Image
import re
from datetime import datetime

images_path = './images/test' # D:/DCIM/100MEDIA

def extract_exif_and_gimbal_yaw(image_path):
    img_exif = Image.open(image_path)
    exif_data = get_exif_data(img_exif)
    lat, lon = get_lat_lon(exif_data)

    timestampImage = exif_data.get('DateTimeOriginal') or exif_data.get('DateTime')
    timestampImage = datetime.strptime(timestampImage, "%Y:%m:%d %H:%M:%S").strftime("%Y-%m-%d_%H-%M-%S")

    with open(image_path, encoding='latin-1') as fd:
        d = fd.read()
        xmp_start = d.find('<x:xmpmeta')
        xmp_end = d.find('</x:xmpmeta')
        xmp_str = d[xmp_start:xmp_end+12]
        match = re.search(r'drone-dji:GimbalYawDegree="([+-]?/d+/.?/d*)"', xmp_str)
        gimbal_yaw = float(match.group(1)) if match else None

    return lat, lon, gimbal_yaw, timestampImage

os.makedirs('./labeled', exist_ok=True)

# Read predictions from JSON file
with open('./predictions_combined.json', 'r') as f:
    predictions = json.load(f)

# Get all .jpg images in the ./detection folder
image_files = glob.glob(f'{images_path}/*.JPG')

image_info = []

# Loop through each image
for img_path in image_files:
    img_name = os.path.basename(img_path)
    image = cv2.imread(img_path)

    for pred in predictions:
        if pred.get('image_id') == img_name[:-4]:
            if pred.get('score') > 0.5 and pred.get('category_id') == 0:
                x, y, w, h = np.array(pred.get('bbox'), dtype=int)
                x_min, y_min = x, y
                x_max, y_max = x + w, y + h

                cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 20)
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                save_path = os.path.join('./labeled', img_name)
                plt.imsave(save_path, image_rgb)

                img_path = os.path.join(f'{images_path}', img_name)

                try:
                    lat, lon, yaw, timestampImage = extract_exif_and_gimbal_yaw(img_path)
                except Exception as e:
                    lat, lon, yaw = None, None, None

                timestampDetection = pred.get('timestampDetected')

                image_info.append({'ImageName': img_name, 'BoundingBox': pred.get('bbox'), 'lat': lat, 'lon': lon, 'yaw': yaw, 'timestampImage': timestampImage, 'timestampDetection': timestampDetection})

info_df = pd.DataFrame(image_info)
info_df.to_excel('image_info.xlsx', index=False)

# Get coordinates of labels

Get coordinates of labels in the format image name, lat, lon and timestamp

The excel file contains the following columns:
- ImageName
- BBoxCenterLat
- BBoxCenterLon
- TimestampImage

In [11]:
import os
from PIL import Image
from draw_utils import get_exif_data, get_lat_lon
import pandas as pd
import numpy as np
import cv2

import matplotlib.pyplot as plt

def get_image_corners(img_width, img_height):
    # Returns corners in image coordinates (centered at image center)
    half_w = img_width / 2
    half_h = img_height / 2
    return np.array([
        [-half_w, -half_h],  # top-left
        [ half_w, -half_h],  # top-right
        [ half_w,  half_h],  # bottom-right
        [-half_w,  half_h],  # bottom-left
    ])

def rotate_points(points, angle_deg):
    # Rotates points by angle_deg (counterclockwise)
    theta = np.deg2rad(angle_deg)
    rot_mat = np.array([
        [np.cos(theta), -np.sin(theta)],
        [np.sin(theta),  np.cos(theta)]
    ])
    return points @ rot_mat.T

def image_corners_to_latlon(center_lat, center_lon, yaw_deg, img_width, img_height, gsd):
    # gsd: ground sample distance in meters per pixel
    # Returns lat/lon for each corner
    # Approximate: assumes flat earth and small area
    corners = get_image_corners(img_width, img_height)
    rotated = rotate_points(corners, yaw_deg)
    # Convert meters to lat/lon offsets
    # 1 deg latitude ~ 111320 m, 1 deg longitude ~ 111320*cos(lat) m
    dlat = rotated[:,1] * gsd / 111320
    dlon = rotated[:,0] * gsd / (111320 * np.cos(np.deg2rad(center_lat)))
    lats = center_lat + dlat
    lons = center_lon + dlon
    return lats, lons

def calculate_bbox_center_latlon(center_lat, center_lon, yaw, img_width, img_height, gsd, bbox_str):
    # Parse bbox string
    x_min, y_min, w, h = [float(v) for v in bbox_str.strip('[]').split(',')]
    x_max = x_min + w
    y_max = y_min + h
    # Get bbox center in image coordinates
    bbox_center = np.array([
        x_min + (x_max - x_min) / 2,
        y_min + (y_max - y_min) / 2
    ])
    # Center at image center
    bbox_center_centered = bbox_center - np.array([img_width/2, img_height/2])
    # Rotate by yaw
    bbox_center_rot = rotate_points(bbox_center_centered.reshape(1, 2), yaw)[0]
    # Convert bbox center to lat/lon
    dlat = bbox_center_rot[1] * gsd / 111320
    dlon = bbox_center_rot[0] * gsd / (111320 * np.cos(np.deg2rad(center_lat)))
    # Adjust sign based on position relative to image center
    if bbox_center[1] < img_height / 2:
        dlat = -dlat
    if bbox_center[0] < img_width / 2:
        dlon = -dlon
    bbox_center_lat = center_lat + dlat
    bbox_center_lon = center_lon + dlon
    return bbox_center_lat, bbox_center_lon

gsd = 0.02
df = pd.read_excel('image_info.xlsx')

# Loop over all images in df
bbox_centers_latlon = []
for idx, row in df.iterrows():
    img_name = row['ImageName']
    bbox_str = row['BoundingBox']
    img_timestamp = row.get('timestampImage', None)

    info_row = info_df[info_df['ImageName'] == img_name]
    if info_row.empty:
        bbox_centers_latlon.append((np.nan, np.nan))
        continue

    center_lat = info_row.iloc[0]['lat']
    center_lon = info_row.iloc[0]['lon']
    yaw = info_row.iloc[0]['yaw']
    img_path = os.path.join('./images/test', img_name) # D:/DCIM/100MEDIA
    img = cv2.imread(img_path)
    img_height, img_width = img.shape[:2]
    bbox_center_lat, bbox_center_lon = calculate_bbox_center_latlon(center_lat, center_lon, yaw, img_width, img_height, gsd, bbox_str)
    bbox_centers_latlon.append((bbox_center_lat, bbox_center_lon, img_timestamp))

bbox_centers_df = pd.DataFrame({
    'ImageName': df['ImageName'],
    'BBoxCenterLat': [lat for lat, lon, _ in bbox_centers_latlon],
    'BBoxCenterLon': [lon for lat, lon, _ in bbox_centers_latlon],
    'TimestampImage': [ts for _, _, ts in bbox_centers_latlon]
})

bbox_centers_df.to_excel('bbox_centers.xlsx', index=False)


# Creating txt files

Based on the bbox_centers excel creating txt files containing lat and lon of the labels

In [ ]:
from datetime import datetime
import os
import pandas as pd

save_path_karlo = 'C:/Users/Stella/AOSeR Dropbox/Stella Dumencic/Cres_experiment/work_folder/monitoring/target_detection'
save_path_stella = './detected_targets'

os.makedirs(save_path_stella, exist_ok=True)

bbox_centers_df = pd.read_excel('bbox_centers.xlsx')

# Save each lat/lon pair to its own file
for index, row in bbox_centers_df.iterrows():
        filename = os.path.join(save_path, f"detected_{index+1:02d}_{row['ImageName'][:-4]}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.txt")
        with open(filename, 'w') as f:
            f.write(f"{row['BBoxCenterLat']} {row['BBoxCenterLon']}/n")

        filename_stella = os.path.join(save_path_stella, f"detected_{index+1:02d}_{row['ImageName'][:-4]}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.txt")
        with open(filename_stella, 'w') as f_stella:
            f_stella.write(f"{row['BBoxCenterLat']} {row['BBoxCenterLon']}/n")

# Converting the bounding box coordinates

Converting the bounding box coordinates from global to local

In [2]:
import numpy as np
from geo_converter import latlon_to_local, local_to_latlon
import pandas as pd
from datetime import datetime
import os

bbox_centers_df = pd.read_excel('bbox_centers.xlsx')

base_point_test = [44.952006, 14.364774]

# Prepare array of all bbox center coordinates
latlon_points = bbox_centers_df[['BBoxCenterLat', 'BBoxCenterLon']].values

# Convert to local coordinates
drifter_location = latlon_to_local(latlon_points, base_point_test)
print(drifter_location)

# Save to npz file with the required key and time_step
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

os.makedirs(f'./marker_advection/target_dirs/target_dir_{timestamp}', exist_ok=True)

for i, point in enumerate(drifter_location):
    np.savez(f'./marker_advection/target_dirs/target_dir_{timestamp}/drifter_measurements_{i+1:02d}_{timestamp}.npz', drifter_location=[point], timestamp=timestamp)

[[-12770.7159669   18470.88507587]
 [-12931.38110713  18427.49642527]
 [-12910.69109766  18445.20550787]
 [-12810.81892227  18544.72852835]
 [-12840.5751839   18471.06866141]
 [-12847.15060902  18494.73862477]
 [-12858.75527845  18481.27717617]
 [-12881.54568116  18476.44791921]
 [-12907.14617244  18506.94332473]
 [-13149.74556779  18521.91958804]
 [-13132.11597309  18451.33676516]
 [-12975.99708755  18514.25412779]
 [-13013.60636843  18555.17838572]
 [-13117.72187348  18573.84889811]
 [-13121.45216757  18577.65655651]
 [-13157.81026163  18629.39138552]
 [-13204.46999669  18622.00237735]
 [-13175.1090539   18620.46601669]
 [-13156.20158635  18615.93546855]
 [-13152.18265261  18616.19295935]
 [-13179.6698128   18574.82057584]
 [-13218.13671039  18630.83040359]
 [-13217.49825542  18632.17863553]
 [-13223.20643598  18630.92631981]
 [-13259.5723416   18633.14243749]]


# Initial advection of the targets

Initial advection of the targets so that their locations at the same time are defined and the continuous advection can start from the same point

In [ ]:
import numpy as np
import os

from marker_advection.advect_initial_marker_locations import advect_initial_marker_locations
from geo_converter import latlon_to_local, local_to_latlon
from datetime import datetime

# C:/Users/Stella/Documents/cres/marker_advection/target_dir
data_dir = f'D:/cres/marker_advection/target_dirs' # measurements in npz format

# C:/Users/Stella/Documents/cres/marker_advection/template_dir/case_Cres
case_dir = f'D:/cres/marker_advection/template_dir/case_Cres' # OpenFOAM case

# C:/Users/Stella/Documents/cres/marker_advection/advection_results_dir
now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
advection_results_dir = f'D:/cres/marker_advection/advection_results_dir/results_initial' # advected marker locations
os.makedirs(advection_results_dir, exist_ok=True)

advection_time_step = 600 # desired advection time step

src_dir = 'C:/Users/Stella/AOSeR Dropbox/Stella Dumencic/Cres_experiment/work_folder/search/flow_fields'
dst_dir = 'D:/cres/marker_advection/template_dir/case_Cres/0'

name = 0

files = sorted(os.listdir(src_dir))

# Save the target locations in the target_dir of that timestamp

target_dirs = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]

if target_dirs:
    target_dirs_sorted = sorted(target_dirs)
    last_target_dir = target_dirs_sorted[-1]

# Go through all drifters in the last target directory
last_target_dir_path = os.path.join(data_dir, last_target_dir)

for index, drifter in enumerate(sorted(os.listdir(last_target_dir_path))):
    if drifter.endswith('.npz'):

        # Calculate the advection time step - now - image timestamp
        data = np.load(os.path.join(last_target_dir_path, drifter))
        drifter_timestamp = data['timestamp']

        drifter_time = datetime.strptime(str(drifter_timestamp)[-8:], "%H-%M-%S")
        now_time = datetime.strptime(now[-8:], "%H-%M-%S")

        time_diff = int((now_time - drifter_time).total_seconds())

        # Find the U file with the nearest timestamp before drifter_timestamp

        nearest_file = None
        drifter_dt = datetime.strptime(str(drifter_timestamp), "%Y-%m-%d_%H-%M-%S")
        min_time_diff = float('inf')
        for f in files:
            try:
                # Extract timestamp from filename (assumes format 'U_YYYY-MM-DD_HH-MM-SS')
                file_ts = f.split('_', 1)[-1]
                file_dt = datetime.strptime(file_ts, "%Y-%m-%d_%H-%M-%S")
                if file_dt <= drifter_dt:
                    diff = (drifter_dt - file_dt).total_seconds()
                    if diff < min_time_diff:
                        min_time_diff = diff
                        nearest_file = f
            except Exception:
                continue

        # if nearest_file:
        #     shutil.copy(os.path.join(src_dir, nearest_file), os.path.join(dst_dir, 'U'))

        # Advect marker locations

        advect_initial_marker_locations(
            case_dir,
            time_diff,
            os.path.join(last_target_dir_path, drifter),
            advection_results_dir,
            index,
            now
        )

npz_files = [f for f in os.listdir(advection_results_dir) if f.endswith('.npz')]

# Collect all drifter locations
all_locations = []
for npz_file in sorted(npz_files):
    data = np.load(os.path.join(advection_results_dir, npz_file))
    loc = data['drifter_location']
    # Ensure loc is 2D (shape: (1, 2) or (2,))
    loc = np.atleast_2d(loc)
    for point in loc:
        all_locations.append(point)

all_locations = np.array(all_locations)

# Save combined locations to a single npz file
name = f'drifter_measurements_{now}.npz'
np.savez(os.path.join(data_dir, f'drifter_measurements_{now}.npz'), drifter_location=all_locations)

# Convert to global coordinates
base_point_test = [44.952006, 14.364774]
latlon_points = all_locations
drifter_location = local_to_latlon(latlon_points, base_point_test)
np.savez(f'{name}_global.npz', drifter_location=drifter_location)

# Save targets to txt files
save_path_karlo = 'C:/Users/Stella/AOSeR Dropbox/Stella Dumencic/Cres_experiment/work_folder/monitoring/target_detection'
save_path_stella = './detected_targets'

os.makedirs(save_path_stella, exist_ok=True)

# Save each lat/lon pair to its own file
for index, point in enumerate(drifter_location):
    filename_karlo = os.path.join(save_path_karlo, f"Detection_{index+1:02d}_{now}.txt")
    with open(filename_karlo, 'w') as f:
        f.write(f"{point[0]} {point[1]}\n")

    filename_stella = os.path.join(save_path_stella, f"Detection_{index+1:02d}_{now}.txt")
    with open(filename_stella, 'w') as f_stella:
        f_stella.write(f"{point[0]} {point[1]}\n")


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:/cres/marker_advection/target_dirs/target_dir_2025-06-01_01-33-24\\target_dir_2025-06-01_01-33-24'

# Converting coordinates

Converts coordinates from local to global

In [ ]:
import numpy as np
from geo_converter import latlon_to_local, local_to_latlon

base_point_test = [44.952006, 14.364774]

# Prepare array of all bbox center coordinates
# Load advected marker locations from the npz file
data = np.load('marker_advection/advection_results_dir/advected_marker_locations_0.npz')
latlon_points = data['drifter_location']

# Convert to local coordinates
drifter_location = local_to_latlon(latlon_points, base_point_test)

# Save to npz file with the required key and time_step
np.savez('bbox_centers_global.npz', time_step=9000, drifter_location=drifter_location)

In [ ]:
from datetime import datetime
import os
import pandas as pd

save_path_karlo = 'C:/Users/Stella/AOSeR Dropbox/Stella Dumencic/Cres_experiment/work_folder/monitoring/target_detection'

os.makedirs(save_path_stella, exist_ok=True)

bbox_centers_df = pd.read_excel('bbox_centers.xlsx')

# Save each lat/lon pair to its own file
for index, row in bbox_centers_df.iterrows():
        filename = os.path.join(save_path, f"detected_{index+1:02d}_{row['ImageName'][:-4]}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.txt")
        with open(filename, 'w') as f:
            f.write(f"{row['BBoxCenterLat']} {row['BBoxCenterLon']}/n")

        filename_stella = os.path.join(save_path_stella, f"detected_{index+1:02d}_{row['ImageName'][:-4]}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.txt")
        with open(filename_stella, 'w') as f_stella:
            f_stella.write(f"{row['BBoxCenterLat']} {row['BBoxCenterLon']}/n")

In [ ]:
from marker_advection.advect_marker_locations import advect_marker_locations
import time
from datetime import datetime

data_dir = f'D:/cres/marker_advection/target_dirs' # measurements in npz format
case_dir = f'D:/cres/marker_advection/template_dir/case_Cres' # OpenFOAM case
advection_results_dir = f'D:/cres/marker_advection/advection_results_dir' # advected marker locations
advection_time_step = 600 # desired advection time step

index = 0

while True:
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    advect_marker_locations(case_dir, advection_time_step, data_dir, advection_results_dir, index+1, timestamp)
    index = index + 1
    time.sleep(120)

# Image map

In [16]:
import folium
import numpy as np

# Get original bbox center points
latlon_points = bbox_centers_df[['BBoxCenterLat', 'BBoxCenterLon']].values

# Load advected (global) points from npz
data_global = np.load('bbox_centers_global.npz')
advected_points = data_global['drifter_location']

# Center map at the mean of all points
all_lats = np.concatenate([latlon_points[:, 0], advected_points[:, 0]])
all_lons = np.concatenate([latlon_points[:, 1], advected_points[:, 1]])
center_lat = all_lats.mean()
center_lon = all_lons.mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=16)

# Add original bbox center points (blue)
for lat, lon in latlon_points:
    folium.CircleMarker(
        location=[lat, lon],
        radius=6,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        popup='Original'
    ).add_to(m)

# Add advected/global points (green)
for lat, lon in advected_points:
    folium.CircleMarker(
        location=[lat, lon],
        radius=6,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        popup='Advected'
    ).add_to(m)

# Draw polylines connecting original and advected points by index
for i in range(min(len(latlon_points), len(advected_points))):
    folium.PolyLine(
        locations=[list(latlon_points[i]), list(advected_points[i])],
        color='purple',
        weight=2,
        opacity=0.8
    ).add_to(m)

m.save('bbox_centers_comparison_map.html')

In [6]:
import folium
import pandas as pd

# Load image info with lat/lon
info_df = pd.read_excel('image_info.xlsx')

# Center map at the mean location
center_lat = info_df['lat'].mean()
center_lon = info_df['lon'].mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=16)

# Add markers for each image
for _, row in info_df.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=row['ImageName'],
        icon=folium.Icon(color='blue', icon='camera', prefix='fa')
    ).add_to(m)

m.save('image_locations_map.html')

# Labels map

In [ ]:
import folium
import pandas as pd

# Load bbox center coordinates
bbox_centers_df = pd.read_excel('bbox_centers.xlsx')

# Center the map at the mean location
center_lat = bbox_centers_df['BBoxCenterLat'].mean()
center_lon = bbox_centers_df['BBoxCenterLon'].mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=16)

# Add markers for each bbox center
for _, row in bbox_centers_df.iterrows():
    folium.Marker(
        location=[row['BBoxCenterLat'], row['BBoxCenterLon']],
        popup=row['ImageName'],
        icon=folium.Icon(color='red', icon='crosshairs', prefix='fa')
    ).add_to(m)

m.save('bbox_centers_map.html')

# Test converting

In [ ]:
from geo_converter import latlon_to_local, local_to_latlon

base_point_test = [44.952006, 14.364774]
point = np.array([[44.952006, 14.364774]])

test_local_point = latlon_to_local(point, base_point_test)

print(test_local_point)

# Test npz file

Local coordinates

In [ ]:
data = np.load('marker_advection/advection_results_dir/advected_marker_locations_0.npz')
print(data.files)
print(data['drifter_location'])

# Test npz file

Global coordinates

In [ ]:
data = np.load('bbox_centers_global.npz')
print(data.files)
print(data['drifter_location'])

# Advection of the targets

Advection for 1 target

In [ ]:
import numpy as np

data = np.load('bbox_centers_local.npz')
print(data.files)
print(data['drifter_location'])
print(data['time_step'])

from marker_advection.advect_marker_locations import advect_marker_locations
import time

# C:/Users/Stella/Documents/cres/marker_advection/target_dir
data_dir = f'D:/cres/marker_advection/target_dir' # measurements in npz format

# C:/Users/Stella/Documents/cres/marker_advection/template_dir/case_Cres
case_dir = f'D:/cres/marker_advection/template_dir/case_Cres' # OpenFOAM case

# C:/Users/Stella/Documents/cres/marker_advection/advection_results_dir
advection_results_dir = f'D:/cres/marker_advection/advection_results_dir' # advected marker locations

advection_time_step = 600 # desired advection time step

gaussian_noise=False

name = 0
advect_marker_locations(case_dir, advection_time_step, data_dir, advection_results_dir, name)
name = name + 1